In [2]:
%matplotlib inline

In [3]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from deap import creator, base, tools, algorithms
import sys

# random sampling
from random import sample

# combination
from itertools import combinations

In [4]:
import warnings

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

#warnings.simplefilter(action='ignore', category=FutureWarning)

In [5]:
random.seed(42)
_functional_relevance_lookup_dict = {}
_population = 1000
_generation = 200
_penalty_weight = 15
_num_one = 6
_mu = 50
_cxpb = 0.8
_mutpb = 0.003

# def trim_newline(s):
#     if s[-1] == '\n':
#         return s[0:-1]
    
#     return s

# workdir = "D://miR_Rank_Feautre_selection//GA_test"
# _functional_relevance_lookup_dict = {}
# with open(workdir + '//FR.txt', 'r') as fh:
#     fh.readline() # skip the first line
#     for line in fh:
#         elements = line.split('\t')
#         _functional_relevance_lookup_dict[elements[0]] = float(trim_newline(elements[1]))

# def functional_relevance_score(items):
#     sum_scores = 0
#     for x in items:
#         try:
#             sum_scores += _functional_relevance_lookup_dict[x]
#         except KeyError:
#             continue
    
#     return sum_scores

def average(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def evaluate(individual, X, y):
#     print (individual)
#     print ("Indivdial: num of 0: %d " % individual.count(0))
#     print ("Indivdial: num of 1: %d " % individual.count(1))
#     print ("The num of genes: %d" % len(individual))
    if(individual.count(0) != len(individual)):
        # 조건 1: 1의 개수가 5보다 클 때, penalty 값을 
        
        # get index with value 0
        cols = [i for i in range(len(individual)) if individual[i] == 1]

        
        #if len(individual) - len(cols) < 5:
        #    return 10000, 10000, 10000
        
        #if len(individual) - len(cols) > 20:
        #    return 10000, 10000, 10000
        
        # get features subset
#         X_parsed = X.drop(X.columns[cols], axis=1)
#         X_subset = pd.get_dummies(X_parsed)

        # apply classification algorithm
        clf = RandomForestClassifier()
        #clf = LogisticRegression()
        
        # fr_score = functional_relevance_score(X_parsed.columns.to_list())
        
        # return average(cross_val_score(clf, X_subset, y, cv=5), scoring = "roc_auc") - _penalty_weight * abs(individual.count(1) - 3), 1/fr_score #, (len(individual) - len(cols)) / len(individual)
        return 100 * average(cross_val_score(clf, X.iloc[:,cols], y, cv=3, scoring='roc_auc')) - _penalty_weight * abs(individual.count(1) - 3), 
#         shuffle = cross_validation.KFold(len(X), n_folds=3, shuffle=True)
#         scores = cross_val_score(rf, X, y, cv=shuffle, scoring='roc_auc')
    else:
        # return 10000, 10000 #, 10000
        return -100000,

In [6]:
workdir = "D://miR_Rank_Feautre_selection//Raw_data"
df_genomic = pd.read_csv(workdir + "//200617_95_samples_miRNA_HC_vs_T2DM_RPM.csv"
                    , index_col = ["id"])
y = df_genomic["Class"] #Class
#X_DN = df_e1.drop(["Class", 'hsa-let-7b-5p', 'hsa-miR-23b-3p', 'hsa-let-7c-5p', 'hsa-miR-30a-3p', 'hsa-miR-200c-3p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-1290'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-1343-3p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-671-3p', 'hsa-miR-3144-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
#X_DN = df_e1.drop(["Class", 'hsa-miR-642a-5p', 'hsa-miR-642a-3p', 'hsa-miR-99b-3p', 'hsa-miR-891a-5p', 'hsa-miR-320c', 'hsa-miR-150-5p'], axis = 1) #Rest dataseet
df_genomic = df_genomic.drop(["Class"], axis = 1) #Rest dataseet
# split data train 70 % and test 30 %
#drop and rotate dataset
df_genomic = df_genomic.T[(df_genomic != 0).any()].T

_number_of_samples = len(df_genomic)
_number_of_items = len(df_genomic.columns.to_list())


In [7]:
len(df_genomic)

95

In [8]:
# 각 miRNA에 대해 각 샘플들에서 가지고 있는 0의 총 비율이 0.9 이상이면 해당 miRNA 제거
cut_off = 0.9
for x in df_genomic.columns:
    try:
        num_zero = df_genomic[x].value_counts(0)[0]
        if num_zero / _number_of_samples > cut_off:
            df_genomic = df_genomic.drop([x], axis = 1)
    except KeyError:
        continue

In [9]:
df_genomic

,hsa-let-7a-3p,hsa-let-7a-5p,hsa-let-7b-3p,hsa-let-7b-5p,hsa-let-7c-5p,hsa-let-7d-3p,hsa-let-7d-5p,hsa-let-7e-3p,hsa-let-7e-5p,hsa-let-7f-1-3p,...,hsa-miR-95-3p,hsa-miR-95-5p,hsa-miR-9-5p,hsa-miR-96-5p,hsa-miR-98-3p,hsa-miR-98-5p,hsa-miR-99a-3p,hsa-miR-99a-5p,hsa-miR-99b-3p,hsa-miR-99b-5p
id,,,,,,,,,,,,,,,,,,,,,
SRR5034616,27.496209,5757.420584,20.354337,572.421088,23.211086,576.170571,606.523529,1.428375,76.060943,2.142562,...,0.178547,0.714187,0.000000,12.319730,16.783401,99.629123,0.357094,28.031850,2.856749,711.330509
SRR5034617,17.487052,12211.478810,13.349737,1326.064677,33.250223,324.103480,710.859702,1.158448,81.863676,2.661673,...,0.179284,0.013791,1.448060,58.529219,7.722988,173.808610,0.055164,11.763766,1.530807,399.830138
SRR5034618,51.480370,44465.243510,25.545184,2590.544879,240.095477,1057.931358,2228.671027,12.626341,1246.741466,4.972536,...,0.292502,0.000000,2.778770,16.233867,34.417748,617.471940,0.731255,14.478854,15.307610,2049.805991
SRR5034619,38.595591,10638.761160,27.037553,698.435723,32.610179,641.264713,901.939748,2.270329,147.364984,4.540658,...,1.031968,0.000000,4.953445,13.209186,21.052141,156.446300,0.206394,31.784604,2.063935,566.550254
SRR5034620,17.703098,3052.382241,18.228933,336.972342,13.671700,402.876946,283.950687,1.139308,41.365656,1.840421,...,0.087639,0.000000,0.525835,11.743640,9.727940,32.514106,0.262917,15.862677,0.964030,219.711226
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SRR7412331,71.385132,78696.617310,30.201402,1902.688337,129.042355,135.906310,1290.423547,2.745582,675.413176,8.236746,...,4.118373,1.372791,164.734921,0.000000,6.863955,2315.898430,0.000000,10.982328,10.982328,404.973347
SRR7412332,62.178990,100443.756800,21.945526,1718.151784,138.988330,69.494165,929.027255,0.000000,429.766545,15.544747,...,1.828794,0.000000,43.891051,0.000000,1.828794,2008.015603,0.914397,51.206227,2.743191,629.105071
SRR7412333,66.274923,102931.718600,22.317066,1836.085884,135.254945,124.434550,1294.389828,0.000000,495.033100,18.935692,...,5.410198,1.352549,58.835901,0.676275,4.057648,2239.145622,0.000000,75.742769,4.057648,3494.311515


In [10]:

# # encode labels column to numbers
# le = LabelEncoder()
# #get label data by index and Encode items into 0 and 1
# le.fit(df_genomic.iloc[:, _labels.index(_target_label)])
# y = le.transform(df_genomic.iloc[:, _labels.index(_target_label)])
#get data excluding labels
X = df_genomic.iloc[:,]

#Min-Max normalization
# X = (X - X.min())/(X.max()-X.min())

# z-normalization
X = (X - X.mean())/X.std()

#X.to_csv('miR.txt', sep='\t', header=True)

In [11]:
creator.create("MyFitnessMulti", base.Fitness, weights=(1.0,)) # Note here <- I used only two weights!  (at first, I tried weights=(-1.0 , -1.0, 1.0)) but it crashes. With deap, you cannot do such a thing.
creator.create("Individual", list, fitness=creator.MyFitnessMulti)

toolbox = base.Toolbox()

#def f(a, b, c): 
#    return np.random.choice(np.arange(a, b), p=c)
  
# A partial function that calls f with 
# a as 3, b as 1 and c as 4. 
#g = partial(f, 0, 2) 




# Attribute generator
def random_individual_three():
    zero_list = [0 for i in range(len(X.columns))]
    index_for_one = np.random.choice(len(X.columns), _num_one ,replace=True)
#     index_for_one = random.sample(range(len(X.columns)),_num_one) # 중복을 허용하지 않는 index random sampling
    # index_for_one = list(combinations(range(len(X.columns)),_num_one)) # combination 생성
    for i in index_for_one:
        zero_list[i]= 1 
    
    print(zero_list.count(1))
    return zero_list
    

# Structure initializers
toolbox.register("rand_ind",random_individual_three)
toolbox.register("individual", tools.initIterate, creator.Individual
                 ,toolbox.rand_ind) #, n=30) #
# toolbox.register("individual", tools.initIterate, creator.Individual, random.sample(range(len(X.columns)),_num_one))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Registration of crossover & mutation functions
#toolbox.register("mate", tools.cxUniform, indpb=0.8)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=_mutpb)

# Registration of selection function
toolbox.register("select", tools.selBest)
# toolbox.register("select", tools.selRoulette)
# toolbox.register("select", tools.selNSGA2) # NSGA-2 applies to multi-objective problems such as knapsack problem
# toolbox.register("select", tools.selTournament, k=1, tournsize=3)

toolbox.register("evaluate", evaluate, X=X, y=y)
logbook = tools.Logbook()

In [12]:
len(X.columns)

1169

In [13]:
# from functools import partial
# random.seed(42)
# gen_idx = random.sample(range(len(X.columns)), _num_one)
# print(gen_idx)
# print(list(gen_idx))

In [14]:
pop = toolbox.population(n=_population)
hof = tools.HallOfFame(1) # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution
# hof = tools.ParetoFront() # a ParetoFront may be used to retrieve the best non dominated individuals of the evolution

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean, axis=0)
stats.register("std", np.std, axis=0)
stats.register("min", np.min, axis=0)
stats.register("max", np.max, axis=0)

# pop, log = algorithms.eaMuPlusLambda(pop, toolbox, _mu, _lambda, _cxpb, _mutpb, 
#                                      ngen=_generation, stats=stats, halloffame=hof) #, verbose=True)

# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, 
#                                   stats=stats, halloffame=hof, verbose=True)

population = []

for i in range(0,_generation + 1):
    print ("Generation: %d" % i)
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=1, stats=stats, halloffame=hof, verbose=True)
    population.append(pop)
# pop, log = algorithms.eaSimple(pop, toolbox, cxpb=_cxpb, mutpb=_mutpb, ngen=_generation, stats=stats, halloffame=hof, verbose=True)

#for ind, fit in zip (pop, )

6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
5
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6


1  	790   	[-6207.21930049]	[24113.7887958] 	[-100000.]	[79.39721098]
Generation: 29
gen	nevals	avg             	std            	min       	max          
0  	0     	[-6207.21930049]	[24113.7887958]	[-100000.]	[79.39721098]
1  	801   	[-6507.00892563]	[24650.82900516]	[-100000.]	[79.39721098]
Generation: 30
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6507.00892563]	[24650.82900516]	[-100000.]	[79.39721098]
1  	772   	[-6607.60276953]	[24826.28009788]	[-100000.]	[87.71180087]
Generation: 31
gen	nevals	avg             	std             	min       	max          
0  	0     	[-6607.60276953]	[24826.28009788]	[-100000.]	[87.71180087]
1  	808   	[-6607.63144924]	[24826.2745007] 	[-100000.]	[84.60788724]
Generation: 32
gen	nevals	avg             	std            	min       	max          
0  	0     	[-6607.63144924]	[24826.2745007]	[-100000.]	[84.60788724]
1  	800   	[-6708.04177313]	[25000.13966002]	[-100000.]	[84.60788724]
Generation: 33
gen	nevals	avg     

1  	804   	[-15820.74354551]	[36465.22096549]	[-100000.]	[76.05713]   
Generation: 66
gen	nevals	avg              	std             	min       	max       
0  	0     	[-15820.74354551]	[36465.22096549]	[-100000.]	[76.05713]
1  	809   	[-15820.88695307]	[36465.16039658]	[-100000.]	[76.99430199]
Generation: 67
gen	nevals	avg              	std             	min       	max          
0  	0     	[-15820.88695307]	[36465.16039658]	[-100000.]	[76.99430199]
1  	810   	[-15821.47655646]	[36464.90615352]	[-100000.]	[80.95666517]
Generation: 68
gen	nevals	avg              	std             	min       	max          
0  	0     	[-15821.47655646]	[36464.90615352]	[-100000.]	[80.95666517]
1  	800   	[-16421.83904483]	[37017.99270017]	[-100000.]	[82.0662768] 
Generation: 69
gen	nevals	avg              	std             	min       	max         
0  	0     	[-16421.83904483]	[37017.99270017]	[-100000.]	[82.0662768]
1  	804   	[-16622.71006898]	[37198.04131274]	[-100000.]	[81.12910481]
Generation: 70
gen	nevals

1  	794   	[-23232.96040936]	[42192.91189768]	[-100000.]	[79.63337832]
Generation: 103
gen	nevals	avg              	std             	min       	max          
0  	0     	[-23232.96040936]	[42192.91189768]	[-100000.]	[79.63337832]
1  	790   	[-23332.64473984]	[42256.36150703]	[-100000.]	[81.09536662]
Generation: 104
gen	nevals	avg              	std             	min       	max          
0  	0     	[-23332.64473984]	[42256.36150703]	[-100000.]	[81.09536662]
1  	818   	[-23333.13022942]	[42256.0939667] 	[-100000.]	[82.18248613]
Generation: 105
gen	nevals	avg              	std            	min       	max          
0  	0     	[-23333.13022942]	[42256.0939667]	[-100000.]	[82.18248613]
1  	798   	[-23533.3961336] 	[42381.50373318]	[-100000.]	[76.37951717]
Generation: 106
gen	nevals	avg             	std             	min       	max          
0  	0     	[-23533.3961336]	[42381.50373318]	[-100000.]	[76.37951717]
1  	790   	[-23634.15914605]	[42443.36407235]	[-100000.]	[81.54520918]
Generation: 107
g

1  	786   	[-28343.9512843] 	[45018.18126114]	[-100000.]	[79.84705353]
Generation: 139
gen	nevals	avg             	std             	min       	max          
0  	0     	[-28343.9512843]	[45018.18126114]	[-100000.]	[79.84705353]
1  	788   	[-28243.86994677]	[44969.98991875]	[-100000.]	[79.84705353]
Generation: 140
gen	nevals	avg              	std             	min       	max          
0  	0     	[-28243.86994677]	[44969.98991875]	[-100000.]	[79.84705353]
1  	797   	[-28344.06689534]	[45018.1107881] 	[-100000.]	[78.68496026]
Generation: 141
gen	nevals	avg              	std            	min       	max          
0  	0     	[-28344.06689534]	[45018.1107881]	[-100000.]	[78.68496026]
1  	796   	[-28544.02831684]	[45113.80093104]	[-100000.]	[85.2751537] 
Generation: 142
gen	nevals	avg              	std             	min       	max         
0  	0     	[-28544.02831684]	[45113.80093104]	[-100000.]	[85.2751537]
1  	796   	[-28644.31979382]	[45161.04212343]	[-100000.]	[79.08981856]
Generation: 143
gen

1  	836   	[-31251.57063803]	[46296.43272072]	[-100000.]	[81.54895786]
Generation: 175
gen	nevals	avg              	std             	min       	max          
0  	0     	[-31251.57063803]	[46296.43272072]	[-100000.]	[81.54895786]
1  	838   	[-31351.70818039]	[46336.71242476]	[-100000.]	[82.29494677]
Generation: 176
gen	nevals	avg              	std             	min       	max          
0  	0     	[-31351.70818039]	[46336.71242476]	[-100000.]	[82.29494677]
1  	793   	[-31552.17825311]	[46416.38171857]	[-100000.]	[80.1694407] 
Generation: 177
gen	nevals	avg              	std             	min       	max         
0  	0     	[-31552.17825311]	[46416.38171857]	[-100000.]	[80.1694407]
1  	810   	[-31751.90643125]	[46495.55721175]	[-100000.]	[81.61268556]
Generation: 178
gen	nevals	avg              	std             	min       	max          
0  	0     	[-31751.90643125]	[46495.55721175]	[-100000.]	[81.61268556]
1  	803   	[-31952.43860474]	[46573.19122081]	[-100000.]	[81.61268556]
Generation: 179

In [15]:
print("Individulas in the hall of fame population")
for x in hof:
    print(x)

Individulas in the hall of fame population
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
print("The # of individuals in the hof population: %d " % len(hof))

The # of individuals in the hof population: 1 


In [73]:
# Best individual 선택
#min_error_rate = 1.0
#for i in hof:
#    if (i.fitness.values[0] < min_error_rate):
#        min_error_rate = i.fitness.values[0]
#        _individual = i
# _individual = tools.selRoulette(hof, 1)[0]
_individual = tools.selBest(hof, 1)[0]
# _individual = tools.selNSGA2(hof, 1)[0]
_individual_features = [X.columns[i] for i in range(len(_individual)) if _individual[i] == 1]
print("The best individual is :" + str(_individual))

The best individual is :[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [74]:
print("The best features are: \t" + str(_individual_features))

The best features are: 	['hsa-miR-29b-1-5p', 'hsa-miR-494-3p', 'hsa-miR-668-3p']


In [75]:
#min_error_rate = 1.0
#for i in hof:
#    if (i.fitness.values[0] < min_error_rate):
#        min_error_rate = i.fitness.values[0]
#        _individual = i

#_individual_features = [X.columns[i] for i in range(len(_individual)) if _individual[i] == 1]
print('Fitness score: \t' + str(_individual.fitness))



Fitness score: 	(87.7118008696956,)


In [76]:
print('Number of Features in Subset: \t' + str(_individual.count(1)))


Number of Features in Subset: 	3


In [77]:
print('Individual: \t' + str(_individual))

Individual: 	[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,

In [78]:
print('Feature Subset\t: ' + str(_individual_features))

Feature Subset	: ['hsa-miR-29b-1-5p', 'hsa-miR-494-3p', 'hsa-miR-668-3p']


In [79]:
_individual.fitness

deap.creator.MyFitnessMulti((87.7118008696956,))

In [80]:
len(_individual_features)

3

In [81]:
for x in _individual_features:
    print ("'{0}', ".format(x), end='')

'hsa-miR-29b-1-5p', 'hsa-miR-494-3p', 'hsa-miR-668-3p', 